In [38]:
import pickle
with open('urls_gyobo.pickle', 'rb') as fr:
    urls = pickle.load(fr)
print(urls.keys())
print(urls)

dict_keys(['소설', '시에세이', '인문', '경제경영', '자기계발', '정치사회', '역사문화', '예술대중문화', '과학'])
{'소설': {'잃어버린 얼굴': {'rank': '951', 'author': '올가 토카르축', 'url': 'https://product.kyobobook.co.kr/detail/S000201512165', 'rate': '8.75', 'count': '2'}, '프랑켄슈타인(초판본)(1818년 오리지널 초판본 표지디자인)': {'rank': '952', 'author': '메리 셸리', 'url': 'https://product.kyobobook.co.kr/detail/S000201743043', 'rate': '0.0', 'count': None}, '표정없는 검사의 분투': {'rank': '953', 'author': '나카야마 시치리', 'url': 'https://product.kyobobook.co.kr/detail/S000201650747', 'rate': '10', 'count': '20'}, '항구의 니쿠코짱!': {'rank': '954', 'author': '니시 가나코', 'url': 'https://product.kyobobook.co.kr/detail/S000201554245', 'rate': '9.74', 'count': '17'}, '허니비': {'rank': '955', 'author': '박문영', 'url': 'https://product.kyobobook.co.kr/detail/S000201621535', 'rate': '10', 'count': '9'}, '서브플롯': {'rank': '956', 'author': '황모과', 'url': 'https://product.kyobobook.co.kr/detail/S000201621568', 'rate': '0.0', 'count': None}, '달의 의지': {'rank': '957', 'author': '황현진', 'url'

In [39]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
pd.DataFrame({'title':'_', 'author':'_', 'genre_1':'_', 'genre_2':'_', 'rank':0, 'rate':0, 'count':0}, index=[0])

,title,author,genre_1,genre_2,rank,rate,count
0,_,_,_,_,0,0,0


In [41]:
class recommBook():
    def __init__(self, gender, age, genres):
        self.ga = 2*(age//10)-2 if gender=='female' else 2*(age//10)-1
        # self.genres = {g1:{'df':df, 'g2_11':df_11, 'g2_12':df_12}}
        self.genres = {g1:{g2:None for g2 in g2s} for g1,g2s in genres.items()}
        for g1 in self.genres.keys():
            self.genres[g1]['df'] = self.call_df(g1)
        print('user info: {}, {}, {}'.format(gender, age, genres))
        
    def call_df(sefl, g1):
        df = pd.read_csv('temp_gyobo_{}.csv'.format(g1))
        print(df.duplicated(['title','author']).sum(), end='->')
        df.drop_duplicates(subset=['title','author'], ignore_index=True, inplace=True)
        print(df.duplicated(['title','author']).sum())
        return df.dropna()
    
    def recomm1(self, g1, g2, n=240, check=True):
        df = self.genres[g1]['df'].copy()
        
        try:
            idx = df[df['genre_2']==g2].index.values[0]
        except:
            check = False
            df = pd.concat([df, pd.DataFrame({'genre_2':[g2]})], ignore_index=True)
            idx = df[df['genre_2']==g2].index.values

        cvect = CountVectorizer(ngram_range=(1,3))
        cvect_g2 = cvect.fit_transform(df['genre_2'])
        csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]
        
        sim_idx = csim_g2[idx, :n].reshape(-1)
        sim_idx = sim_idx if check else sim_idx[sim_idx != idx]
        self.genres[g1][g2] = df.iloc[sim_idx].sort_values(['brank','rate'], ascending=[True, False])
        return self.genres[g1][g2]
    
    def recomm1_result(self):
        select_dfs = []
        g1_n, g1_c = 240//len(self.genres.keys()), 12//len(self.genres.keys())
        print('g1_n, g1_c', g1_n, g1_c)
        for g1, g2s in self.genres.items():
            g2_n, g2_c = g1_n//(len(g2s.keys())-1), g1_c//(len(g2s.keys())-1)
            print('g2_n, g2_c', g2_n, g2_c)
            for g2 in g2s.keys():
                if g2=='df': continue
                select_dfs.append(self.recomm1(g1, g2, g2_n).iloc[:g2_c,:])
        return pd.concat(select_dfs, axis=0)
    
    def recomm2_rate(self, g1, g2, n=240, check=True):
        df = self.genres[g1]['df'].copy()
        
        cvect = CountVectorizer(ngram_range=(1,3))
        cvect_g2 = cvect.fit_transform(df['genre_2'])
        csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]
        
        idx = df[df['title'] == title].index.values
        sim_idx = csim_g2[idx, :n].reshape(-1)
        sim_idx = sim_idx[sim_idx != idx]
        return df.iloc[sim_idx].sort_values(['rate', 'brank'], ascending=[False, True])

In [42]:
gender, age = 'male', 20
genres = {'소설': ['장르소설','영미소설'],
          '시에세이': ['테마에세이']} # g1:[g2_1, g2_2]
myrecomm = recommBook(gender, age, genres)
# myrecomm.recomm1('소설', '한국소설').iloc[:2,:]
myrecomm.recomm1_result()

0->0
0->0
user info: male, 20, {'소설': ['장르소설', '영미소설'], '시에세이': ['테마에세이']}
g1_n, g1_c 120 6
g2_n, g2_c 60 3
g2_n, g2_c 120 6


,title,author,genre_1,genre_2,brank,rate,count
693,해리 포터와 비밀의 방 1(해리포터 20주년 개정판),J. K. 롤링,"소설, 소설, 소설, 소설,","영미소설, 판타지소설, 영미소설, 드라마/영화소설, 장르소설, 판타지소설, 장르소설...",214.0,9.84,155.0
707,해리 포터와 비밀의 방 2(해리포터 20주년 개정판),J. K. 롤링,"소설, 소설, 소설, 소설,","영미소설, 판타지소설, 영미소설, 드라마/영화소설, 장르소설, 판타지소설, 장르소설...",233.0,9.86,160.0
597,눈물을 마시는 새 세트,이영도,"소설, 소설,","한국소설, 판타지소설, 장르소설, 판타지소설,",311.0,9.63,61.0
693,해리 포터와 비밀의 방 1(해리포터 20주년 개정판),J. K. 롤링,"소설, 소설, 소설, 소설,","영미소설, 판타지소설, 영미소설, 드라마/영화소설, 장르소설, 판타지소설, 장르소설...",214.0,9.84,155.0
640,낭만적 연애와 그 후의 일상,알랭 드 보통,"소설,","영미소설, 영미소설일반,",256.0,9.05,486.0
670,우주를 듣는 소년,루스 오제키,"소설,","영미소설, 영미소설일반,",290.0,10.00,27.0
882,너의 하루가 시원하길 바라(‘너의 하루가 따숩길 바라’ 썸머 에디션),고은지,"시/에세이, 시/에세이,","테마에세이, 그림에세이, 나라별 에세이, 한국에세이,",7.0,9.87,116.0
892,아기 판다 푸바오,에버랜드 동물원,"시/에세이, 시/에세이, 시/에세이,","테마에세이, 동물등장에세이, 테마에세이, 포토에세이, 나라별 에세이, 한국에세이,",17.0,10.00,38.0
916,사랑할 기회,박근호,"시/에세이, 시/에세이,","테마에세이, 사랑/연애에세이, 나라별 에세이, 한국에세이,",42.0,9.95,30.0
918,망그러진 만화,유랑,"시/에세이, 시/에세이,","테마에세이, 그림에세이, 나라별 에세이, 한국에세이,",44.0,9.78,158.0
